In [1]:
# Secondary user multicast beamforming
#   minimize ||x||^2
#   subject to |h_i^H x|^2 >= tau
#              |g_i^H x|^2 <= eta
# with variable x in complex^n.
# Data vectors h_i and g_i are also in complex^n.
# The script below expands out the complex part and
# works with real numbers only.
# Refer to the companion paper for the details of
# the rewriting.

import numpy as np
import cvxpy as cvx
from qcqp import *
import scipy 

In [2]:
# n = 50
# m = 20
# l = 5

n = 20
m = 5
l = 2

tau = 20
eta = 2

np.random.seed(1)
HR = np.random.randn(m, n)
HI = np.random.randn(m, n)
A = np.hstack((HR, HI))
B = np.hstack((-HI, HR))

GR = np.random.randn(l, n)
GI = np.random.randn(l, n)
C = np.hstack((GR, GI))
D = np.hstack((-GI, GR))

x = cvx.Variable(2*n)
# obj = cvx.Minimize(cvx.sum_squares(x))
# cons = [
#     cvx.square(A*x) + cvx.square(B*x) >= tau,
#     cvx.square(C*x) + cvx.square(D*x) <= eta
# ]

sni_f0 = (scipy.sparse.csr_matrix(np.eye(2*n)), scipy.sparse.csc_matrix((2*n,1)), 0)
sni_fs = []
for i in range(m):
    sni_fs += [(-scipy.sparse.csr_matrix(A[i:i+1].T @ A[i:i+1] + B[i:i+1].T @ B[i:i+1]), scipy.sparse.csc_matrix((2*n, 1)), tau, "<=")]
for i in range(l):
    sni_fs += [(scipy.sparse.csr_matrix(C[i:i+1].T @ C[i:i+1] + D[i:i+1].T @ D[i:i+1]), scipy.sparse.csc_matrix((2*n, 1)), -eta, "<=")]

prob = QCQPProblem(sni_f0, sni_fs, x)

qcqp = QCQP(prob)

# sample from the semidefinite relaxation
qcqp.suggest(SDR)
print("SDR-based lower bound: %.3f" % qcqp.sdr_bound)

# f_cd, v_cd = qcqp.improve(COORD_DESCENT)
# print("Coordinate descent: objective %.3f, violation %.3f" % (f_cd, v_cd))
# f_cd, v_cd = qcqp.improve(ADMM, rho=np.sqrt(m+l), phase1=False)
# print("Coordinate descent: objective %.3f, violation %.3f" % (f_cd, v_cd))

# # SDR solution is cached and not solved again
# qcqp.suggest(SDR)
f_dccp, v_dccp = qcqp.improve(DCCP)
print("Penalty CCP: objective %.3f, violation %.3f" % (f_dccp, v_dccp))

qcqp.suggest(SDR)
print("SDR lower bound: %.4f" % qcqp.sdr_bound)
f_ipopt, v_ipopt = qcqp.improve(COORD_DESCENT)
f_ipopt, v_ipopt = qcqp.improve(IPOPT, verbose=False)
x_ipopt = x.value
print("Coordinate descent + ipopt: objective %.4f, violation %.4f" % (f_ipopt, v_ipopt))

qcqp.suggest(SDR)
print("SDR lower bound: %.4f" % qcqp.sdr_bound)
f_ipopt, v_ipopt = qcqp.improve(IPOPT, verbose=False)
x_ipopt = x.value
print("Ipopt: objective %.4f, violation %.4f" % (f_ipopt, v_ipopt))

qcqp.suggest(SDR)
f_admm, v_admm = qcqp.improve(COORD_DESCENT)
print("Coordinate descent: objective %.3f, violation %.3f" % (f_admm, v_admm))
f_admm, v_admm = qcqp.improve(ADMM, rho=np.sqrt(m+l))
print("Coordinate descent + ADMM: objective %.3f, violation %.3f" % (f_admm, v_admm))
f_admm, v_admm = qcqp.improve(COORD_DESCENT, phase1=False)
print("Coordinate descent + ADMM + coordinate descent: objective %.3f, violation %.3f" % (f_admm, v_admm))


|x| = 40
SDR-based lower bound: 1.970
Penalty CCP: objective 1.970, violation 0.000
SDR lower bound: 1.9698
Coordinate descent + ipopt: objective 1.9698, violation 0.0000
SDR lower bound: 1.9698
Ipopt: objective 1.9698, violation 0.0000
Coordinate descent: objective 2.558, violation 0.000
Coordinate descent + ADMM: objective 2.306, violation 0.000
Coordinate descent + ADMM + coordinate descent: objective 2.240, violation 0.000
